In [1]:
#import libraries needed for this proces
import requests
from datetime import datetime
import numpy
import time
import json
import pandas as pd
import sys

In [2]:
#function to create an API request based on URL and  parameters
def get_request(url, parameters=None):
    print("Your request is being processed.")
    response = requests.get(url=url, params=parameters)
    if response:
        return response.json()
        
    else:
        print("Request unsuccessful, retrying in 5 seconds.")
        time.sleep(5)
        return get_request(url, parameters)
    time.sleep(1)

In [3]:
#using the function to get all data id's which include appid and app name 
url = "https://api.steampowered.com/ISteamApps/GetAppList/v2/"
parameters = {"request": all}
data_ids = get_request(url, parameters=parameters)
print("Your request has been processed successfully, call upon 'data_ids' to preview the full list of IDs and names of all currently available products on Steam as of ", datetime.now().strftime("%d/%m/%Y %H:%M:%S"), ".")

Your request is being processed.
Your request has been processed successfully, call upon 'data_ids' to preview the full list of IDs and names of all currently available products on Steam as of  16/03/2022 21:10:24 .


In [4]:
#Code to collect all app id's in one library
app_ids = []
print("Creating a list of all IDs currently available on Steam")
for item in data_ids['applist']['apps']:
    app_ids.append(item['appid'])

print("List created sucessfully, the total number of IDs available on Steam is:", len(app_ids),".")
print("Call upon 'app_ids' to preview the full list of extracted IDs.")

Creating a list of all IDs currently available on Steam
List created sucessfully, the total number of IDs available on Steam is: 138251 .
Call upon 'app_ids' to preview the full list of extracted IDs.


In [5]:
# example code that collects the details of 1 app by appid  
url = "http://store.steampowered.com/api/appdetails/"
parameters = {"appids": "1815720"}
app_data = get_request(url, parameters=parameters)
app_data["1815720"]['data']


Your request is being processed.


{'type': 'game',
 'name': 'Split Tactics',
 'steam_appid': 1815720,
 'required_age': 0,
 'is_free': False,
 'detailed_description': 'Split Tactics is a casual game about choices. There are a total of 13 elements in this game. The 6 basic elements are: Fire, Ice, Poison, Dark, Light and Curse. Then there are 7 more that are their combinations: Chaos (Fire+Dark), Reflection (Ice+Light), Toxic Mist (Poison+Dark), Madness (Dark+Curse), Fume Gas (Fire+Poison), Horror Breath (Ice+Dark) and Divine Flame (Light+Fire). Each element basic or not, has its own effects when applied to an enemy. Players should choose spells based on effectiveness to creatures and also to avoid making opponent the next most powerful mage. A good strategic balance is required to overcome opponent and also stay alive for the duration of the game.<h2 class="bb_tag">Start of the Game</h2>Each player draws 5 cards from the reward card pile. The first enemy or trap is opened from the explore card pile. <br><br><img src="ht

In [7]:
## collecting the data with a for loop that checks if the response was a succes = True. and than takes the data attribute 
dataframe = pd.DataFrame()
dataframe_categories = pd.DataFrame()
amount_app_ids = 10
counter = 1   

for i in app_ids: 
    url = "http://store.steampowered.com/api/appdetails/"
    parameters = {"appids": {i}}
    app_data = get_request(url, parameters=parameters)
    id = str(i)
    print("Processing data pertaining to Steam ID:"+id)
    response = app_data[id]
    succes = response["success"]
    if (succes): 
        data_json = response["data"]
        steam_appid = data_json["steam_appid"]
        result = pd.json_normalize(data_json)
        dataframe = pd.concat([dataframe, result],ignore_index= True)
        try:
            categories = pd.json_normalize(data_json,record_path = ['categories'], meta = ["steam_appid"])
            dataframe_categories = pd.concat([dataframe_categories,categories],ignore_index= True)
        except:
            pass
        counter = counter + 1 
    if (counter > amount_app_ids):
        break           


Your request is being processed.
Processing data pertaining to Steam ID:216938
Your request is being processed.
Processing data pertaining to Steam ID:660010
Your request is being processed.
Processing data pertaining to Steam ID:660130
Your request is being processed.
Processing data pertaining to Steam ID:1118314
Your request is being processed.
Processing data pertaining to Steam ID:1275822
Your request is being processed.
Processing data pertaining to Steam ID:1343832
Your request is being processed.
Processing data pertaining to Steam ID:1828741
Your request is being processed.
Processing data pertaining to Steam ID:662172
Your request is being processed.
Processing data pertaining to Steam ID:1360782
Your request is being processed.
Processing data pertaining to Steam ID:1820332
Your request is being processed.
Processing data pertaining to Steam ID:1815690
Your request is being processed.
Processing data pertaining to Steam ID:1815720
Your request is being processed.
Processing 

In [13]:
#write the dataframes to an xlsx file

dataframe.to_excel("..steam-API\\data\\raw_df.xlsx")
dataframe_categories.to_excel("..steam-API\\data\\categories_df.xlsx")

FileNotFoundError: [Errno 2] No such file or directory: '..steam-API\\data\\raw_df.xlsx'

In [12]:
### example to create a dictionary that checks if the id is already in the dictionary and then if not in dictionary get data 

"""
details = {}

for i in app_ids:
  id = str(i)
  if id not in details:
     // get data

     if(success):
       // put data in details
       details[id] = data

"""     

'\ndetails = {}\n\nfor i in app_ids:\n  id = str(i)\n  if id not in details:\n     // get data\n\n     if(success):\n       // put data in details\n       details[id] = data\n\n'